In [10]:
import os
import sys
sys.path.append("C:/users/kira-/documents/projet_data/end-to-end-ml-flow/src")


In [11]:
%pwd

'c:\\Users\\kira-\\Documents\\projet_data\\end-to-end-ml-flow\\research'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir:Path

In [13]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [14]:
os.chdir('../')

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [23]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def dowload_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} dowload! with folowing info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [ ]:
try:
    manag_config=ConfigurationManager()
    configM=manag_config.get_data_ingestion_config()
    data_ing=DataIngestion(configM)
    data_ing.dowload_file()
    data_ing.extract_zip_file()
    except Exception as e :
        raise e

[2025-07-04 13:40:07,861: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-04 13:40:07,863: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-04 13:40:07,865: INFO: common: yaml file: schema.yaml  loaded successfully]
[2025-07-04 13:40:07,868: INFO: common: created directory at: artifacts]
[2025-07-04 13:40:07,870: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-04 13:40:09,574: INFO: 3698930113: artifacts/data_ingestion/data.zip dowload! with folowing info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9148:14C420:25E693:2A42C2:6867BD97
Accept-Ranges: bytes
Date: Fri, 04 